In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import warnings
from dataclasses import dataclass
from enum import StrEnum
import random

from torch.optim import AdamW

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from lib import *

from transformers import get_cosine_schedule_with_warmup

from sklearn.model_selection import KFold

import pickle

In [2]:

DEBUG = False
DATA_DIR = "./processed-data/sagittal_t2_stir_segmentation"
MODEL_NAME = "tf_efficientnet_b0.ns_jft_in1k" if DEBUG else "tf_efficientnet_b5.ns_jft_in1k"
DEVICE = "cuda:0"
MODEL_DIR = "./models/sagittial_t2_stir"

EPOCHS = 15

HEIGHT = 512
WIDTH = 512

# Network params
N_CLASSES = 5
HIDDEN_DIM = 768
LOCATION_DIM = 10


SEED = 8620
N_WORKERS=4

GRAD_ACC = 1
TGT_BATCH_SIZE = 16
BATCH_SIZE = TGT_BATCH_SIZE // GRAD_ACC
EARLY_STOPPING_EPOCH = 3
OUTPUT_DIR = "models/sagittial_t2_stir"


TEST_BATCH_SIZE = 1

N_FOLDS = 5

LR = 2e-4 * TGT_BATCH_SIZE / 32
AUG = True

In [3]:
df = pd.read_csv('./data/train.csv')

PATH = f"./processed-data/sagittal_t2_stir_segmentation/"
patient_ids = [*map(lambda p: int(os.path.basename(p).split(".")[0]), glob.glob(f"{PATH}/*.pkl"))]

df = df[df["study_id"].isin(patient_ids)]


In [4]:

def vertical_flip(img, coord):
    new_coord = coord.copy()
    img = cv2.flip(img, 0)  # 0 for vertical flip
    new_coord[:, 1] = img.shape[1] - new_coord[:, 1]  # Adjusting the y-coordinates for vertical flip
    return img, new_coord

def horizontal_flip(img, coord):
    new_coord = coord.copy()
    img = cv2.flip(img, 1)  # Corrected to 1 for horizontal flip
    new_coord[:, 0] = img.shape[0] - new_coord[:, 0]  # Adjusting the x-coordinates for horizontal flip
    return img, new_coord


In [5]:
class RSNA24Dataset(Dataset):
    def __init__(self, patient_ids, transformations=[], phase="train", positive_negative_ratio=0.5, positive_augment_prob=0.25, negative_augment_prob=0.15):
        self.patient_ids = patient_ids
        self.transformations = transformations
        self.positive_negative_ratio = positive_negative_ratio
        self.positive_augment_prob = positive_augment_prob
        self.negative_augment_prob = negative_augment_prob
        self.phase = phase
    
    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, idx):
        
        patient_id = self.patient_ids[idx]

        patient_info_path = f"{DATA_DIR}/{patient_id}.pkl"

        with open(patient_info_path, "rb") as f:
            data = pickle.load(f)

        sagittal_t2_stir = [*data["series"].values()]

        if len(sagittal_t2_stir) == 0:
            return None
        else:
            sagittal_t2_stir = sagittal_t2_stir[0]


        if self.phase == "train":
            
            y_class = np.zeros(5)
            y_loc = np.zeros((5,2))
            
            if random.random() <= self.positive_negative_ratio:
                imgs = [img for img in sagittal_t2_stir["images"] if len(img["labels"]) != 0]
                has_sampled_postive_label = True
            else:
                imgs = [img for img in sagittal_t2_stir["images"] if len(img["labels"]) == 0]
                has_sampled_postive_label = False
                
            img =  random.choice(imgs) if len(imgs) != 0 else random.choice(sagittal_t2_stir["images"])
            x = img["img"].astype(np.float32)
    
            for label in img["labels"]:
                y_loc[label.level - 1] = np.array([label.x, label.y])
                y_class[label.level - 1] = 1
    
            aug_prob = self.positive_augment_prob if has_sampled_postive_label else self.negative_augment_prob
            x,y_loc = self.transform(x, y_class, y_loc,aug_prob)
    
            x = x / 255
            x = np.expand_dims(x, 0)
    
            return x, (y_class,y_loc)
            
        else:
            imgs = sagittal_t2_stir["images"]
            num_imgs = len(imgs)
            xs = np.zeros((num_imgs, HEIGHT, WIDTH))
            
            y_classes = np.zeros((num_imgs, 5))
            y_locs = np.zeros((num_imgs, 5, 2))

            for idx,img in enumerate(imgs):
                xs[idx] = img["img"].astype(np.float32)
                
                for label in img["labels"]:
                    y_locs[idx, label.level - 1] = np.array([label.x, label.y])
                    y_classes[idx, label.level - 1] = 1
            xs = xs / 255
            xs = np.expand_dims(xs, 1)
            return xs, (y_classes, y_locs)
            


    def transform(self, x, y_class, y_loc, aug_prob):
        for transformation in self.transformations:
            if random.random() <= aug_prob:
                x,y_loc = transformation(x,y_loc)
                y_loc = np.expand_dims(y_class, 1) * y_loc
                    
        return x, y_loc
            

In [6]:


class AvgPool(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):

        r = list(x.shape)[:-2]
        r.append(-1)
        x = x.reshape(tuple(r)).mean(dim=-1)
        
        return x

class RSNA24Model(nn.Module):
    def __init__(self, model_name, n_classes=N_CLASSES, location_dim=LOCATION_DIM, hidden_dim=HIDDEN_DIM, pretrained=True, features_only=True):
        super().__init__()
        
        self.img_features  = 512
        self.img_features_dim = 2048
        self.hidden_dim = hidden_dim
        self.n_classes = n_classes
        self.location_dim = location_dim

        self.initial_conv = nn.Conv2d(1,3, (3,3), stride=(1,1), padding=(1,1))
        self.feature_extractor =  timm.create_model( model_name , pretrained=pretrained , features_only=features_only, out_indices=[-1] )

        self.predictors = nn.Sequential(
            nn.Conv2d(self.img_features, self.img_features, (3,3), stride=(1,1), padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(self.img_features, self.img_features_dim, (1,1), stride=(1,1), bias=False),
            nn.BatchNorm2d(self.img_features_dim),
            AvgPool(),
            nn.Linear(self.img_features_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, self.n_classes)
        )

        self.location_predictor = nn.Sequential(
            nn.Conv2d(self.img_features, self.img_features, (3,3), stride=(1,1), padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(self.img_features, self.img_features, (3,3), stride=(1,1), padding=(1,1)),
            nn.SiLU(),
            nn.Conv2d(self.img_features, self.img_features_dim, (1,1), stride=(1,1), bias=False),
            nn.BatchNorm2d(self.img_features_dim),
            AvgPool(),
            nn.Linear(self.img_features_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, self.location_dim)
        )

    
    def forward(self, x, freeze_conv=False):
        x = self.initial_conv(x)

        if freeze_conv:
            with torch.no_grad():
                x = self.feature_extractor(x)[0]
        else:
            x = self.feature_extractor(x)[0]
            
        class_preds = self.predictors(x)
        location_preds = self.location_predictor(x)
        return class_preds, location_preds

In [7]:

bce = nn.BCEWithLogitsLoss()
def criterion(y_class, y_loc, pred_class, pred_loc, alpha=0.01):
    
    class_loss = bce(pred_class, y_class)
    
    loc_loss = (((pred_loc - y_loc) ** 2).sum(-1).sqrt() * y_class).mean(-1).mean()

    # deviation_loss = ((pred_loc[:, :-1, :] - pred_loc[:, 1:, :]) ** 2).sum(dim=-1).sqrt()
    # deviation_loss = (-1 * deviation_loss).exp().sum(dim=-1).mean()
    
    return (class_loss + alpha * loc_loss), {"class_loss": class_loss, "loc_loss": loc_loss}


def train(model,dataloader, criterion, optimizer, scheduler=None, freeze_conv=False):
    
    model.train()
    optimizer.zero_grad()
    training_loss = []
    
    for idx, (x, y) in enumerate(pbar := tqdm(dataloader)):
        
        x, y_class, y_loc = x.to(DEVICE), y[0].to(DEVICE), y[1].to(DEVICE)
        
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            pred_class, pred_loc = model(x, freeze_conv=freeze_conv)
            pred_loc = pred_loc.reshape(-1,5,2)
            loss, _ = criterion(y_class, y_loc, pred_class, pred_loc)
            loss = loss / GRAD_ACC
            
        loss.backward()
        norm = nn.utils.clip_grad_norm(model.parameters(), 1.0)

        training_loss.append(loss.item() * GRAD_ACC)

        if (idx + 1) % GRAD_ACC == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()


        pbar.set_description(f"loss: {loss.item() * GRAD_ACC:.6f}, | norm: {norm:.4f}| training_loss: {np.mean(training_loss):.6f}")
        
    
    return np.mean(training_loss)

def validation(model, data_loader, criterion):
    
        model.eval()
    
        val_loss = []
        loc_loss = []
        class_loss = []

        with torch.no_grad():
    
            for idx, (x, y) in enumerate(pbar := tqdm(data_loader)):
                
                x, y_class, y_loc = x.to(DEVICE).squeeze(0).float(), y[0].to(DEVICE).squeeze(0), y[1].to(DEVICE).squeeze(0)

                with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                    pred_class, pred_loc = model(x)
                    pred_loc = pred_loc.reshape(-1,5,2)
                    loss, per_cat_loss = criterion(y_class, y_loc, pred_class, pred_loc)

                val_loss.append(loss.item())
                loc_loss.append(per_cat_loss["loc_loss"].item())
                class_loss.append(per_cat_loss["class_loss"].item())
                
    
                pbar.set_description(f"current loss: {loss.item():.6f}, validation_loss: {np.mean(val_loss):.6f}, loc_loss: {np.mean(loc_loss)}, class_loss: {np.mean(class_loss)}")

        return np.mean(val_loss), np.mean(loc_loss), np.mean(class_loss)
    


In [8]:
skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

In [9]:



for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(df)))):
    print('#'*30)
    print(f'start fold{fold}')
    print('#'*30)
    print(len(trn_idx), len(val_idx))
    
    df_train = df.iloc[trn_idx]
    df_valid = df.iloc[val_idx]
    
    train_ds = RSNA24Dataset(df_train["study_id"].unique(), phase='train', transformations=[vertical_flip, horizontal_flip])
    train_dl = DataLoader(
                train_ds,
                batch_size=BATCH_SIZE,
                shuffle=True,
                pin_memory=True,
                drop_last=True,
                num_workers=N_WORKERS
                )

    valid_ds = RSNA24Dataset(df_valid["study_id"].unique(), phase='valid', transformations=[])
    valid_dl = DataLoader(
                valid_ds,
                batch_size=1,
                shuffle=False,
                pin_memory=True,
                drop_last=False,
                num_workers=N_WORKERS
                )
    
    model = RSNA24Model(MODEL_NAME).to(DEVICE)
    optimizer = AdamW(model.parameters(), lr=LR)

    
    warmup_steps = EPOCHS/10 * len(train_dl) // GRAD_ACC
    num_total_steps = EPOCHS * len(train_dl) // GRAD_ACC
    num_cycles = 0.475
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps=warmup_steps,
                                                num_training_steps=num_total_steps,
                                                num_cycles=num_cycles)

    best_val_loss   = 1.04
    best_loc_loss   = 23.73
    best_class_loss = 0.80
    early_stop_counter = 0

    for epoch in range(1, EPOCHS + 1):
        print(f"EPOCH: {epoch}")
        _ = train(model, train_dl, criterion, optimizer, scheduler, freeze_conv=False)
        val_loss, loc_loss, class_loss = validation(model, valid_dl, criterion)

        print(f"val_loss: {val_loss:.4f} | loc_loss: {loc_loss:.4f} | class_loss: {class_loss:.4f}")

        if loc_loss < best_loc_loss:
            early_stop_counter = 0
            print(f"updating best_loc_loss from {best_loc_loss} to {loc_loss}")
            
            best_val_loss = val_loss
            best_loc_loss = loc_loss
            best_class_loss = class_loss

            print(f"updated losses: {best_val_loss=}, {best_loc_loss=}, {best_class_loss=}")

            print("Saving model....")
            fname = f'{OUTPUT_DIR}/best_loc_model_fold-{fold}.pt'
            torch.save(model.state_dict(), fname)
            
        else:
            early_stop_counter += 1
            print(f"{EARLY_STOPPING_EPOCH - early_stop_counter} more epochs to train until early stopping")

        if early_stop_counter == EARLY_STOPPING_EPOCH:
            break
        

##############################
start fold0
##############################
1579 395


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


EPOCH: 1


  0%|                                                                                                                                                                                                                  | 0/98 [00:00<?, ?it/s]/tmp/ipykernel_90767/202607252.py:31: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  norm = nn.utils.clip_grad_norm(model.parameters(), 1.0)
loss: 1.387324, | norm: 0.9872| training_loss: 2.145542: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:16<00:00,  5.88it/s]
current loss: 0.353810, validation_loss: 0.355342, loc_loss: 21.4514547046671, class_loss: 0.14082731581196722: 100%|███████████████████████████████████████████████████████████████████████████████████████| 395/395 [00:17<00:00, 22.11it/s]


val_loss: 0.3553 | loc_loss: 21.4515 | class_loss: 0.1408
updating best_loc_loss from 23.73 to 21.4514547046671
updated losses: best_val_loss=0.35534186285863817, best_loc_loss=21.4514547046671, best_class_loss=0.14082731581196722
Saving model....
EPOCH: 2


...

val_loss: 0.1153 | loc_loss: 2.1901 | class_loss: 0.0934
2 more epochs to train until early stopping
##############################
start fold2
##############################
1579 395


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


EPOCH: 1


loss: 2.316605, | norm: 0.5060| training_loss: 2.299715:  12%|█████████████████▋                                                                                                                              | 12/98 [00:02<00:16,  5.28it/s]


KeyboardInterrupt: 